# Installs

In [68]:
%%sh
pip3.9 install plotly > /dev/null
pip3.9 install numpy > /dev/null
pip3.9 install seaborn > /dev/null

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

# Import canisters

In [69]:
import json
import os
root = os.path.dirname(os.getcwd())

with open(root+'/.dfx/local/canister_ids.json') as f:
    canisters = json.load(f)
print("canister list:", canisters)
data_asset_canister = canisters['data_assets']['local']
print("canister id:", data_asset_canister)
data_asset_did = root+'/.dfx/local/canisters/data_assets/data_assets.did'
print("canister did:", data_asset_did)

canister list: {'__Candid_UI': {'local': 'bw4dl-smaaa-aaaaa-qaacq-cai'}, 'dapp_frontend': {'local': 'bkyz2-fmaaa-aaaaa-qaaaq-cai'}, 'data_assets': {'local': 'bd3sg-teaaa-aaaaa-qaaba-cai'}, 'fractional_NFT': {'local': 'be2us-64aaa-aaaaa-qaabq-cai'}, 'sales_contract': {'local': 'br5f7-7uaaa-aaaaa-qaaca-cai'}}
canister id: bd3sg-teaaa-aaaaa-qaaba-cai
canister did: /Users/vincentlopez/projects/riderlab/dapp/.dfx/local/canisters/data_assets/data_assets.did


# Build identity and IC agent

In [70]:
from ic.client import Client
from ic.identity import Identity
from ic.agent import Agent
# Identity and Client are dependencies of Agent
identity = Identity()
# client = Client(url = "https://ic0.app")
client = Client(url = "http://127.0.0.1:8000")
agent = Agent(identity, client)
datasetId = 1

# Pull dataset configuration and field map

In [71]:
from ic.canister import Canister

# read canister candid from file
canister_did = open(data_asset_did).read()
# create a data_assets canister instance
data_asset = Canister(agent=agent, canister_id=data_asset_canister, candid=canister_did)
# call canister method with instance
datasetConfig = data_asset.getDatasetByDatasetId(1)[0][0]
# build column mapping for panda: dimension_id->title
configClean = {}
for dim in datasetConfig['dimensions']:
    configClean[dim['dimension_id']] = dim['title']
print(configClean)

{0: 'gender', 1: 'ethnicity', 2: 'parental_education', 3: 'lunch', 4: 'preparation_course', 5: 'math_score', 6: 'reading_score', 7: 'writing_score'}


# Pull dataset and prepare records for panda

In [72]:
query1 = data_asset.getDatasetDownload(1, ['we5ZuowTbz7arQLsDe'])[0]["Ok"]

def flatteNestedType(row):
    new_record = {
        "id": row["id"]["Id"],
        "created_at": row["created_at"],
        "updated_at": row["updated_at"]
    }
    for item in row["values"]:
        key = configClean[item['dimension_id']]
        val = item['value']['Metric'] if 'Metric' in item['value'].keys() else item['value']['Attribute']
        new_record[key] = val
    return new_record
datasetClean = list(map(flatteNestedType, query1))

In [73]:
# properly store dataset dynamically
# - not a (incorrectly formatted) file
# - preserves types
import pandas as pd
data = pd.DataFrame(datasetClean)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1000 non-null   int64 
 1   created_at     1000 non-null   int64 
 2   updated_at     1000 non-null   int64 
 3   gender         1000 non-null   object
 4   ethnicity      1000 non-null   object
 5   reading_score  1000 non-null   int64 
 6   writing_score  1000 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 54.8+ KB


,id,created_at,updated_at,gender,ethnicity,reading_score,writing_score
0,999,1683089747342076000,1683089747342076000,male,group A,6000,6200
1,998,1683089747342076000,1683089747342076000,female,group C,7400,8200
2,997,1683089747342076000,1683089747342076000,female,group C,3500,4100
3,996,1683089747342076000,1683089747342076000,male,group D,9100,9200
4,995,1683089747342076000,1683089747342076000,male,group C,7000,6500


In [74]:
import plotly.express as px
fig2 = px.pie(data, values = data['ethnicity'].value_counts().values, names = data['ethnicity'].value_counts().index)
fig2.show()

In [75]:
import seaborn as sns
sns.scatterplot(x = data.math_score, y = data.reading_score, hue = data.writing_score)

AttributeError: 'DataFrame' object has no attribute 'math_score'

In [ ]:
query2 = data_asset.getAnalytics({
        'dataset_id' : datasetId,
        'attributes' : [0,1],
        'metrics' : [6,7],
        'filters' : [],
    }, ['we5ZuowTbz7arQLsDe'])[0]["Ok"]

print(query2['counts'])

pandaReady = []
for group in query2['analytics']:
    res = {"group_key": group['group_key'], "count": group['count']}
    for metric in group['metrics']:
        key = configClean[metric[0]]
        val = metric[1]/100/group['count']
        res[key] = val
    pandaReady.append(res)

data = pd.DataFrame(pandaReady)
data.head()

# Dynamic Datasets and Producers

In [82]:
print("Principal:", data_asset.myUser())
print("Dataset producers:", data_asset.getProducers(datasetId))
print("Is user a producer:", data_asset.isUserProducer(datasetId))

Principal: [Principal(k36pn-57oun-i5wts-6j5g2-5yngs-khmo6-nkbm6-2k2vs-a2fur-tdmfq-dqe)]
Dataset producers: [[[{'id': Principal(2vxsx-fae), 'is_enabled': True, 'created_at': 1683089726526779000}, {'id': Principal(k36pn-57oun-i5wts-6j5g2-5yngs-khmo6-nkbm6-2k2vs-a2fur-tdmfq-dqe), 'is_enabled': True, 'created_at': 1683202476700187000}]]]
Is user a producer: [True]


In [92]:
import csv

with open(root+'/script/exams.csv', mode ='r')as file:
    csvFile = csv.reader(file)
    data = []
    for line in csvFile:
        values = []
        for idx, val  in enumerate(line):
            values.append({
                'dimension_id': idx,
                'value': {"Metric": val} if isinstance(val, int) else {"Attribute": val}

            })
        data.append({
            'id': {'Id': 10000+csvFile.line_num},
            'values': values
        })


data_asset.putManyEntries(1, data)

[]

In [ ]:
# logging
# - dataset usage, cost analysis
# - incident analysis at record level

# Data deletion
# - right to be forgotten, GDPR
# - corrupted datapoint(s)


In [ ]:
# next steps:
# - loader
# - encryption
# - rust for analytics
# - http outcalls to AWS/ML web2